In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [5]:
mimic_iv_path = "/data/wang/junh/datasets/physionet.org/files/mimiciv/2.2"
mm_dir = "/data/wang/junh/datasets/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")

In [6]:
include_notes = True
include_cxr = True
include_ecg = True
standard_scale = True
include_missing = False

In [28]:
ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_new.pkl"))
ireg_vitals_ts_df = ireg_vitals_ts_df.drop(columns=["hosp_time_delta"])
ireg_vitals_ts_df.rename(columns={'icu_time_delta': 'timedelta'}, inplace=True)
imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals_new.pkl"))

# ireg_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_icu.pkl"))
# imputed_vitals = pd.read_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals_icu.pkl"))
print(ireg_vitals_ts_df.columns)
print(imputed_vitals.columns)

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['timedelta'] >= 0]
imputed_vitals = imputed_vitals[imputed_vitals['timedelta'] >= 0]

Index(['subject_id', 'hadm_id', 'stay_id', 'timedelta', 'Anion Gap',
       'Bicarbonate', 'Calcium, Total', 'Chloride', 'Creatinine',
       'Diastolic BP', 'GCS - Eye Opening', 'GCS - Motor Response',
       'GCS - Verbal Response', 'Glucose', 'Heart Rate', 'Hematocrit',
       'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Mean BP',
       'Neutrophils', 'O2 Saturation', 'Phosphate', 'Platelet Count', 'RDW',
       'Red Blood Cells', 'Respiratory Rate', 'Sodium', 'Systolic BP',
       'Urea Nitrogen', 'Vancomycin', 'White Blood Cells'],
      dtype='object')
Index(['subject_id', 'hadm_id', 'stay_id', 'timedelta', 'Anion Gap',
       'Bicarbonate', 'Calcium, Total', 'Chloride', 'Creatinine',
       'Diastolic BP', 'GCS - Eye Opening', 'GCS - Motor Response',
       'GCS - Verbal Response', 'Glucose', 'Heart Rate', 'Hematocrit',
       'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Mean BP',
       'Neutrophils', 'O2 Saturation', 'Phosphate', 'Platelet Count', 'RDW',
       'Red

In [29]:
print(len(ireg_vitals_ts_df.columns))

34


In [32]:
if include_notes:
    notes_df = pd.read_pickle(os.path.join(output_dir, "icu_notes_text_embeddings.pkl"))
    print(notes_df.columns)
    notes_df = notes_df[notes_df['stay_id'].notnull()]

    notes_df = notes_df[notes_df['icu_time_delta'] >= 0]

if include_cxr:
    cxr_df = pd.read_pickle(os.path.join(output_dir, "cxr_embeddings_stay.pkl"))
    print(cxr_df.columns)
    cxr_df = cxr_df[cxr_df['icu_time_delta'] >= 0]

if include_ecg:
    ecg_df = pd.read_pickle(os.path.join(output_dir, "ecg_embeddings_icu.pkl"))
    print(ecg_df.columns)
    ecg_df = ecg_df[ecg_df['icu_time_delta'] >= 0]

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text', 'stay_id', 'icu_time_delta',
       'hosp_time_delta', 'biobert_embeddings'],
      dtype='object')
Index(['dicom_id', 'subject_id', 'study_id',
       'PerformedProcedureStepDescription', 'ViewPosition',
       'ProcedureCodeSequence_CodeMeaning', 'ViewCodeSequence_CodeMeaning',
       'PatientOrientationCodeSequence_CodeMeaning', 'densefeatures',
       'predictions', 'cxrtime', 'hadm_id', 'stay_id', 'icu_time_delta',
       'hosp_time_delta'],
      dtype='object')
Index(['subject_id', 'hadm_id', 'stay_id', 'icu_time_delta', 'ecg_time',
       'path', 'embeddings'],
      dtype='object')


In [33]:
print("Number of stays in ecg: ", ecg_df['stay_id'].nunique())
print("Number of stays in cxr: ", cxr_df['stay_id'].nunique())
print("Number of stays in notes: ", notes_df['stay_id'].nunique())

Number of stays in ecg:  35925
Number of stays in cxr:  14568
Number of stays in notes:  56824


In [34]:
icustays_df = pd.read_csv(os.path.join(mimic_iv_path, "icu", "icustays.csv.gz"), low_memory=False)
icustays_df['intime'] = pd.to_datetime(icustays_df['intime'])
icustays_df['outtime'] = pd.to_datetime(icustays_df['outtime'])

In [35]:
valid_stay_ids = icustays_df['stay_id'].unique()

ireg_vitals_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(valid_stay_ids)]
imputed_vitals = imputed_vitals[imputed_vitals['stay_id'].isin(valid_stay_ids)]

if include_notes:
    notes_df = notes_df[notes_df['stay_id'].isin(valid_stay_ids)]

if include_cxr:
    cxr_df = cxr_df[cxr_df['stay_id'].isin(valid_stay_ids)]

if include_ecg:
    ecg_df = ecg_df[ecg_df['stay_id'].isin(valid_stay_ids)]    

In [36]:
print("Number of stays in ecg: ", ecg_df['stay_id'].nunique())
print("Number of stays in cxr: ", cxr_df['stay_id'].nunique())
print("Number of stays in notes: ", notes_df['stay_id'].nunique())

Number of stays in ecg:  35925
Number of stays in cxr:  14568
Number of stays in notes:  56824


In [37]:

if not include_missing:
    unique_stays = ireg_vitals_ts_df['stay_id'].unique()
    print(f"Number of stays with vitals: {len(unique_stays)}")

    if include_notes:
        unique_stays = np.intersect1d(unique_stays, notes_df['stay_id'].unique())
        print(f"Number of stays with notes: {len(unique_stays)}")

    if include_cxr:
        unique_stays = np.intersect1d(unique_stays, cxr_df['stay_id'].unique())
        print(f"Number of stays with cxr: {len(unique_stays)}")

    if include_ecg:
        unique_stays = np.intersect1d(unique_stays, ecg_df['stay_id'].unique())
        print(f"Number of stays with ecg: {len(unique_stays)}")        
else:
    unique_stays = ireg_vitals_ts_df['stay_id'].unique()
    print(f"Number of stays with vitals: {len(unique_stays)}")

    if include_notes:
        # Get stays with either TS or notes
        unique_stays = np.union1d(unique_stays, notes_df['stay_id'].unique())
        print(f"Number of stays with either TS or notes: {len(unique_stays)}")
    
    if include_cxr:
        unique_stays = np.union1d(unique_stays, cxr_df['stay_id'].unique())
        print(f"Number of stays with either TS, notes, cxr: {len(unique_stays)}")

    if include_ecg:
        unique_stays = np.union1d(unique_stays, ecg_df['stay_id'].unique())
        print(f"Number of stays with either TS, notes, cxr, ecg: {len(unique_stays)}")        

Number of stays with vitals: 73172
Number of stays with notes: 56822
Number of stays with cxr: 14540
Number of stays with ecg: 9070


In [38]:
if include_ecg:
    unique_stays = np.union1d(unique_stays, ecg_df['stay_id'].unique())
    print(f"Number of stays with either TS, notes, cxr, ecg: {len(unique_stays)}") 

Number of stays with either TS, notes, cxr, ecg: 35925


In [39]:
# Create train, val, test splits
np.random.seed(0)
np.random.shuffle(unique_stays)
train_stays = unique_stays[:int(0.7*len(unique_stays))]
val_stays = unique_stays[int(0.7*len(unique_stays)):int(0.85*len(unique_stays))]
test_stays = unique_stays[int(0.85*len(unique_stays)):]

In [40]:
train_ireg_ts_df = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'].isin(train_stays)].copy()
train_imputed_df = imputed_vitals[imputed_vitals['stay_id'].isin(train_stays)].copy()

cols = train_ireg_ts_df.columns.tolist()
cols = [col for col in cols if col not in ['subject_id', 'hadm_id', 'stay_id', 'timedelta']]

if standard_scale:
    for col in cols:
        scaler = StandardScaler()
        scaler.fit(train_ireg_ts_df[[col]])
        ireg_vitals_ts_df[col] = scaler.transform(ireg_vitals_ts_df[[col]])

        scaler = StandardScaler()
        scaler.fit(train_imputed_df[[col]])
        imputed_vitals[col] = scaler.transform(imputed_vitals[[col]])

In [42]:
import yaml

f_path = "/data/wang/junh/datasets/multimodal/hcup_ccs_2015_definitions.yaml"
with open(f_path, 'r') as f:
    hcup_ccs = yaml.safe_load(f)


In [43]:
f_path = "/data/wang/junh/datasets/multimodal/icd10cmtoicd9gem.csv"
icd10_to_icd9_df = pd.read_csv(f_path, low_memory=False)


In [44]:
benchmark_diags = {}

i = 0
for key in hcup_ccs.keys():
    curr_entry = hcup_ccs[key]

    if not curr_entry['use_in_benchmark']:
        continue
    
    curr_entry['icd9'] = curr_entry['codes']

    icd10_codes = []
    for code in curr_entry['codes']:
        curr_icd10_codes = icd10_to_icd9_df[icd10_to_icd9_df['icd9cm'] == code]['icd10cm'].values

        for icd10_code in curr_icd10_codes:
            icd10_codes.append(icd10_code)

    curr_entry['icd10'] = icd10_codes

    # Drop codes from curr_entry
    curr_entry.pop('codes')

    curr_entry['id'] = i
    benchmark_diags[key] = curr_entry
    i += 1

In [45]:
admissions_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "admissions.csv.gz"))
admissions_df = admissions_df.rename(columns={"hospital_expire_flag": "died"})
admissions_df = admissions_df[["subject_id", "hadm_id", "died"]]

In [46]:
d_icd_diagnoses = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "d_icd_diagnoses.csv.gz"))
diagnoses_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "diagnoses_icd.csv.gz"))

diagnoses_df = diagnoses_df.merge(d_icd_diagnoses, on=["icd_code", 'icd_version'], how="left")

In [ ]:
def get_stay_list(stays):
    stays_list = []

    for curr_stay in tqdm(stays):
        curr_stay_ireg = ireg_vitals_ts_df[ireg_vitals_ts_df['stay_id'] == curr_stay].copy()
        curr_stay_imputed = imputed_vitals[imputed_vitals['stay_id'] == curr_stay].copy()

        if len(curr_stay_ireg) == 0:
            continue

        if include_notes:
            curr_stay_notes = notes_df[notes_df['stay_id'] == curr_stay].copy()

        if include_cxr:
            curr_stay_cxr = cxr_df[cxr_df['stay_id'] == curr_stay].copy()

        curr_stay_dict = {}
        curr_stay_dict['name'] = curr_stay_ireg['subject_id'].iloc[0]
        curr_stay_dict['hadm_id'] = curr_stay_ireg['hadm_id'].iloc[0]
        curr_stay_dict['stay_id'] = curr_stay
        curr_stay_dict['ts_tt'] = curr_stay_ireg['timedelta'].values

        curr_stay_ireg.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
        ireg_ts_mask = curr_stay_ireg.notnull()
        curr_stay_ireg.fillna(0, inplace=True)
        curr_stay_dict['irg_ts'] = curr_stay_ireg.values
        if curr_stay_ireg.isna():
            print("reg", subject_id, hadm_id, stay_id)
        curr_stay_dict['irg_ts_mask'] = ireg_ts_mask.values.astype(int)

        curr_stay_imputed.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'timedelta'], inplace=True)
        if curr_stay_imputed.isna():
            print("imputed", subject_id, hadm_id, stay_id)
        curr_stay_dict['reg_ts'] = curr_stay_imputed.values

        if include_notes:
            if len(curr_stay_notes) == 0:
                curr_stay_dict['text_data'] = []
                curr_stay_dict['text_time_to_end'] = []
                curr_stay_dict['text_missing'] = 1
                curr_stay_dict['text_embeddings'] = []
            else:
                curr_stay_dict['text_data'] = curr_stay_notes['text'].tolist()
                curr_stay_dict['text_time_to_end'] = curr_stay_notes['icu_time_delta'].values
                curr_stay_dict['text_embeddings'] = [emb[0][0] for emb in curr_stay_notes['biobert_embeddings']]
                curr_stay_dict['text_missing'] = 0
        
        if include_cxr:
            if len(curr_stay_cxr) == 0:
                curr_stay_dict['cxr_feats'] = []
                curr_stay_dict['cxr_time'] = []
                curr_stay_dict['cxr_missing'] = 1
            else:
                curr_stay_dict['cxr_feats'] = curr_stay_cxr['densefeatures'].tolist()
                curr_stay_dict['cxr_time'] = curr_stay_cxr['timedicu_time_deltaelta'].values
                curr_stay_dict['cxr_missing'] = 0

        if include_ecg:
            curr_stay_ecg = ecg_df[ecg_df['stay_id'] == curr_stay].copy()
            if len(curr_stay_ecg) == 0:
                curr_stay_dict['ecg_feats'] = []
                curr_stay_dict['ecg_time'] = []
                curr_stay_dict['ecg_missing'] = 1
            else:
                curr_stay_dict['ecg_feats'] = curr_stay_ecg['embeddings'].tolist()
                curr_stay_dict['ecg_time'] = curr_stay_ecg['timedelta'].values
                curr_stay_dict['ecg_missing'] = 0            

        curr_diagnoses = diagnoses_df[diagnoses_df['hadm_id'] == curr_stay_dict['hadm_id']]

        curr_labels = np.zeros(len(benchmark_diags.keys()))

        for index, row in curr_diagnoses.iterrows():
            for key in benchmark_diags.keys():
                curr_bench_diag = benchmark_diags[key]
                if (row['icd_version'] == 9) and (row['icd_code'] in curr_bench_diag['icd9']):
                    curr_labels[curr_bench_diag['id']] = 1
                elif (row['icd_version'] == 10) and (row['icd_code'] in curr_bench_diag['icd10']):
                    curr_labels[curr_bench_diag['id']] = 1

        curr_stay_dict['label'] = curr_labels

        stays_list.append(curr_stay_dict)

    return stays_list

train_stays_list = get_stay_list(train_stays)
val_stays_list = get_stay_list(val_stays)
test_stays_list = get_stay_list(test_stays)

 33%|███▎      | 8421/25147 [1:03:43<2:06:38,  2.20it/s]

In [ ]:
# Save the data
import pickle

base_name = "pheno"

base_name += "-all"

if include_cxr:
    if include_notes:
        base_name += "-cxr-notes"
    else:
        base_name += "-cxr"

if include_ecg:
    base_name += "-ecg"
    
if include_missing:
    base_name += "-missingInd"

f_path = os.path.join(output_dir, f"train_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving train stays to {f_path}")
    pickle.dump(train_stays_list, f)

f_path = os.path.join(output_dir, f"val_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving val stays to {f_path}")
    pickle.dump(val_stays_list, f)

f_path = os.path.join(output_dir, f"test_{base_name}_stays.pkl")
with open(f_path, 'wb') as f:
    print(f"Saving test stays to {f_path}")
    pickle.dump(test_stays_list, f)

Saving train stays to /data/wang/junh/datasets/multimodal/preprocessing/train_pheno-all-cxr-notes-ecg_stays.pkl
Saving val stays to /data/wang/junh/datasets/multimodal/preprocessing/val_pheno-all-cxr-notes-ecg_stays.pkl
Saving test stays to /data/wang/junh/datasets/multimodal/preprocessing/test_pheno-all-cxr-notes-ecg_stays.pkl


In [4]:
import pickle

def load_pickle(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

# Assuming the path to your pickle file
train_pickle_path = '/data/wang/junh/datasets/multimodal/preprocessing/train_pheno-all-cxr-notes-ecg_stays.pkl'
# val_pickle_path = '/data/wang/junh/datasets/multimodal/preprocessing/val_pheno-all-cxr-notes-ecg_stays.pkl'
# test_pickle_path = '/data/wang/junh/datasets/multimodal/preprocessing/test_pheno-all-cxr-notes-ecg_stays.pkl'

train_data = load_pickle(train_pickle_path)
# val_data = load_pickle(val_pickle_path)
# test_data = load_pickle(test_pickle_path)

In [7]:
def check_data_completeness(data):
    for i, stay in enumerate(data):
        if stay['reg_ts'].all() == 0:
            print(f"Missing data in stay index {i}, stay_id: {stay['stay_id']}")

# Check completeness for the first few entries
check_data_completeness(train_data)
